In [10]:
import os

from mysql import connector
from dotenv import load_dotenv

# Access environment variables
load_dotenv()

NAME = os.getenv("DB_NAME")
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
IP = os.getenv("DB_IP")
PORT = os.getenv("DB_PORT")

In [11]:
class Database:
    _connection = None  # 클래스 변수로 연결 상태 관리

    @staticmethod
    def get_connection():
        if Database._connection is None or not Database._connection.is_connected():
            Database._connection = connector.connect(
                database = NAME,
                user = USER,
                password = PASSWORD,
                host = IP,
                port = PORT
            )
        return Database._connection

In [12]:
def execute(query, values=None):
    try:
        conn = Database.get_connection()
        cursor = conn.cursor()

        # values가 있으면 쿼리와 함께 전달
        if values:
            cursor.execute(query, values)
        else:
            cursor.execute(query)

        rows = cursor.fetchall()
        for row in rows:
            print(row)

        cursor.close()
    except connector.Error as e:
        print(f"Error: {e}")

In [13]:
def classify_ddc(ddc_code):
    
    ddc_str = str(ddc_code)
    main_part = ddc_str.split(".")[0]
    
    # 대분류와 중분류 생성
    major_class = f"{main_part[0]}00"
    minor_class = f"{main_part[:2].ljust(2, '0')}0"
    
    return {"large_class": major_class, "middle_class": minor_class}

In [ ]:
def update_book_counts(ddc_code):
    try:
        # DDC 코드로 대분류와 중분류 계산
        ddc_classes = classify_ddc(ddc_code)
        large_class = ddc_classes["large_class"]
        middle_class = ddc_classes["middle_class"]

        # 대분류 테이블 업데이트
        large_query = """
        INSERT INTO large_classification (category, book_count)
        VALUES (%s, 1)
        ON DUPLICATE KEY UPDATE book_count = book_count + 1;
        """
        execute(large_query, (large_class,))

        # 중분류 테이블 업데이트
        middle_query = """
        INSERT INTO middle_classification (category, book_count)
        VALUES (%s, 1)
        ON DUPLICATE KEY UPDATE book_count = book_count + 1;
        """
        execute(middle_query, (middle_class,))

        # 커밋하여 변경 사항 저장
        Database.get_connection().commit()
        print(f"Updated counts for large_class: {large_class}, middle_class: {middle_class}")
    except Exception as e:
        print(f"Error updating book counts: {e}")